### Segment Geospatial (samgeo)

In [1]:
#conda install pytorch torchvision -c pytorch
#pip install segment-geospatial groundingdino-py leafmap localtileserver
#conda install gdal


In [1]:
from samgeo import tms_to_geotiff, split_raster
from samgeo.text_sam import LangSAM
import leafmap


In [10]:
# imagem local

image = "../1_estudo_segmentar_cores/DJI_0128.tif"

In [4]:
# dividir em pedaços menors, lidar com restrições de memória

split_raster(image, out_dir="tiles", tile_size=(1000, 1000), overlap=0)

In [16]:
# carregar o modelo de segmentação

sam = LangSAM(model_type='vit_l')

final text_encoder_type: bert-base-uncased


In [6]:
# prompt de texto do objeto a ser detectado

text_prompt = "human"

In [7]:
mascaras = sam.predict_batch(
    images='tiles',
    out_dir='masks',
    text_prompt=text_prompt,
    box_threshold=0.1,
    text_threshold=0.1,
    mask_multiplier=255,
    dtype='uint8',
    merge=True,
    verbose=True,
)

Processing image 01 of 49: tiles/tile_0_0.tif...
No objects found in the image.
Processing image 02 of 49: tiles/tile_0_1.tif...
No objects found in the image.
Processing image 03 of 49: tiles/tile_0_2.tif...
Processing image 04 of 49: tiles/tile_0_3.tif...
Processing image 05 of 49: tiles/tile_0_4.tif...
No objects found in the image.
Processing image 06 of 49: tiles/tile_0_5.tif...
No objects found in the image.
Processing image 07 of 49: tiles/tile_0_6.tif...
No objects found in the image.
Processing image 08 of 49: tiles/tile_1_0.tif...
No objects found in the image.
Processing image 09 of 49: tiles/tile_1_1.tif...
Processing image 10 of 49: tiles/tile_1_2.tif...
Processing image 11 of 49: tiles/tile_1_3.tif...
No objects found in the image.
Processing image 12 of 49: tiles/tile_1_4.tif...
Processing image 13 of 49: tiles/tile_1_5.tif...
No objects found in the image.
Processing image 14 of 49: tiles/tile_1_6.tif...
No objects found in the image.
Processing image 15 of 49: tiles/ti

In [11]:
# criar um mapa para visualizar os resultados

m = leafmap.Map(center=[-27.4349, -48.3917], zoom=17, height="500px")
# m.add_basemap("SATELLITE")
m.add_raster(image, nodata=0, layer_name='Img')
m.add_layer_manager()

In [14]:
# adicionar a máscara final a visualização

m.add_raster('masks/merged.tif', cmap='summer', nodata=0, layer_name='Mask')
#m

Map(bottom=311009239.0, center=[-27.435140809036554, -48.3915701508522], controls=(ZoomControl(options=['posit…

In [17]:
# exportar a máscara para um shapefile

sam.raster_to_vector("masks/merged.tif", "output/merged.shp")

In [19]:
style = {
    "color": "#3388ff",
    "weight": 2,
    "fillColor": "#7c4185",
    "fillOpacity": 0.5,
}
m.add_vector("output/merged.shp", layer_name="Vector", style=style)
m

Map(bottom=311009108.0, center=[-27.43482479071428, -48.39175455272198], controls=(ZoomControl(options=['posit…